In [2]:
import cv2
import numpy as np
from dataloader import Trial

In [3]:
video_path = r'..\data\TH8\TH8ST17A3D250TS0_2.mov'
recording = Trial(video_path)

In [4]:
recording.gauss_blur(inplace=True, sigmaX=1, sigmaY=1)
recording.gauss_blur(inplace=True, sigmaX=1, sigmaY=1)
recording.edges(inplace=True, thresh1=20, thresh2=100)

In [ ]:
def create_kalman_filter(x, y):
    kf = cv2.KalmanFilter(4, 2)
    kf.transitionMatrix = np.array([
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ], np.float32)

    kf.measurementMatrix = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
    ], np.float32)

    kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-4
    kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1e-1
    kf.errorCovPost = np.eye(4, dtype=np.float32)
    kf.statePost = np.zeros((4, 1), np.float32)
    return kf

def distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

kalman_filters = []
missed_counts = []


lower_color = np.array([0, 10, 0])
upper_color = np.array([20, 180, 50])

MAX_DISTANCE = 50
MAX_MISSED_FRAMES = 100


for frame in list(recording[:]):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # measurement = None
    detections = []

    for cnt in contours:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        if radius > 10:
            detections.append((int(x), int(y)))

    predictions = []
    
    for kf in kalman_filters:
        pred=kf.predict()
        predictions.append((int(pred[0][0]), int(pred[1][0])))

    used_detections = set()
    used_filters = set()

    for i, pred in enumerate(predictions):
        best_match = None
        best_dist = MAX_DISTANCE
        for j, det in enumerate(detections):
            if j in used_detections:
                continue
            d = distance(pred, det)
            if d < best_dist:
                best_match = j
                best_dist = d
            if best_match is not None:
                kf = kalman_filters[i]
                meas = np.array([[np.float32(detections[best_match][0])],
                                 [np.flaot32(detections[best_match][1])]])
                
                kf.correct(meas)
                used_detections.add(best_match)
                used_filters.add(i)
                missed_counts[i] = 0

    for j, det in enumerate(detections):
        if j not in used_detections:
            kalman_filters.append(create_kalman_filter(det[0], det[1]))
            missed_counts.append(0)

    for i in range(len(kalman_filters)):
        if i not in used_filters:
            missed_counts[i] += 1

    for i in reversed(range(len(kalman_filters))):
        if missed_counts[i] > MAX_MISSED_FRAMES:
            kalman_filters.pop(i)
            missed_counts.pop(i)

    for det in detections:
        cv2.circle(frame, det, 6, (0, 0, 255), -1)

    for pred in predictions:
        cv2.circle(frame, pred, 6, (0, 255, 0), 2)

    cv2.imshow("Mult...", frame)
    if cv2.waitKey(10) & 0xFF == 27:
        break

cv2.destroyAllWindows()
    

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-15:Bad number of channels) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xb60a28d2::Set<3,-1,-1>,struct cv::impl::A0xb60a28d2::Set<0,5,-1>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
